In [11]:

import sys
sys.path.append('../src/')

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import config
import itertools
import pickle

from tqdm import tqdm
from pathlib import Path
from sklearn.model_selection import train_test_split
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from utils import brier_score_tensor, make_ndarray_from_csv, impf_make_ndarray_from_csv
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score 
from torch.nn.functional import softmax, one_hot
from Dataset import CNS
from torch.utils.data import DataLoader
from utils import get_int_label

In [12]:
dat_cfg = config.data_config
clf_cfg = config.classifier_config
impf_cfg = config.impf_config

In [13]:
group = 'Embryonal'
fold = '1.1'
outer_fold = '1.0'
impf_alg = 'MLP'
with open(os.path.join(impf_cfg['IMPORTANT_FEATURES_DIR'], impf_alg, group, f'{outer_fold}_combined.pkl'), 'rb') as file:
    impf = pickle.load(file)    

In [14]:
print(len(impf))

29


In [15]:
df_fold_train = pd.read_csv(os.path.join(dat_cfg['TRAIN_CSV_DIR'], group, f'{fold}_train.csv'))
df_outer_fold_train = pd.read_csv(os.path.join(dat_cfg['TRAIN_CSV_DIR'], group, f'{outer_fold}_train.csv'))
df_fold_train = df_fold_train.loc[:, impf]
df_outer_fold_train = df_outer_fold_train.loc[:, impf]

C:\Users\OA\AppData\Local\Temp/ipykernel_6008/2710884814.py:3: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_fold_train = df_fold_train.loc[:, impf]
C:\Users\OA\AppData\Local\Temp/ipykernel_6008/2710884814.py:4: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_outer_fold_train = df_outer_fold_train.loc[:, impf]


In [16]:
len(impf), len(df_fold_train.columns), len(df_outer_fold_train.columns)

(29, 29, 29)

In [17]:
train_features, train_labels, val_features, val_labels = impf_make_ndarray_from_csv(group, fold, impf)
train_labels_int = np.array([get_int_label(label, group) for label in train_labels])
val_labels_int = np.array([get_int_label(label, group) for label in val_labels])
train_dataset = CNS(train_features, train_labels_int, mode = 'train')
val_dataset = CNS(val_features, val_labels_int, mode = 'val')
train_loader = DataLoader(train_dataset, batch_size = impf_cfg['mlp_train_batch_size'], shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = impf_cfg['mlp_val_batch_size'], shuffle = False)

../src\utils.py:206: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  train_features = np.array(df_train.loc[:,impf])
../src\utils.py:209: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  test_features = np.array(df_test.loc[:,impf])


In [19]:
len(train_labels_int), len(val_labels_int)

(449, 124)

In [20]:
test_features, test_labels = impf_make_ndarray_from_csv(group, outer_fold, impf, mode = 'test')
test_labels_int = np.array([get_int_label(label, group) for label in test_labels])
test_dataset = CNS(test_features, test_labels_int, mode = 'val')
test_loader = DataLoader(test_dataset, batch_size = impf_cfg['mlp_val_batch_size'], shuffle = False)

../src\utils.py:178: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  test_features = np.array(df_test.loc[:,impf])


In [21]:
len(val_loader)

8